In [4]:
import os

import sys
import os


from datetime import datetime, timedelta
import taxi_demand_predictor.config as cfg
import logging
import pandas as pd
from typing import List
from taxi_demand_predictor.data import save_data
from taxi_demand_predictor.pipeline import retrieve_data_in_range, validate_and_save_data, transform_to_ts_data
import hopsworks


In [ ]:
# Fetch data from the API for the last 28 days
current_date = pd.to_datetime(datetime.utcnow()).floor('H')
end = current_date - timedelta(days=7*10)
start = end - timedelta(days=7*52)

# Define the base URL for the API
base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/"

start, end

In [ ]:
# STREAM_DATA_DIR_RAW = RAW_DATA_DIR / "2023-2024"
# STREAM_DATA_DIR_BRONZE = BRONZE_DATA_DIR / "2023-2024"
# STREAM_DATA_DIR_SILVER = SILVER_DATA_DIR / "2023-2024"
# STREAM_DATA_DIR_GOLD = GOLD_DATA_DIR / "2023-2024"
# INPUT_RAW_DIR = RAW_DATA_DIR / "2023-2024"/f'yellow_tripdata_{start.year}-{start.month:02d}_to_{end.year}-{end.month:02d}.parquet'
# INPUT_SILVER_DIR = STREAM_DATA_DIR_SILVER /f'ts_data_{start.year}-{start.month:02d}_to_{end.year}-{end.month:02d}.parquet'

In [ ]:
# Example usage
base_path = "/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/data/raw"

df = retrieve_data_in_range(start, end, RAW_DATA_DIR)
df

In [ ]:
save_data(df, STREAM_DATA_DIR_RAW, f'yellow_tripdata_{start.year}-{start.month:02d}_to_{end.year}-{end.month:02d}.parquet')

In [ ]:
validated = validate_and_save_data(STREAM_DATA_DIR_RAW, start, end)

save_data(validated, STREAM_DATA_DIR_BRONZE, f'validated_yellow_tripdata_{start.year}-{start.month:02d}_to_{end.year}-{end.month:02d}.parquet')

In [ ]:
validated = pd.read_parquet(f'{STREAM_DATA_DIR_BRONZE}/validated_yellow_tripdata_{start.year}-{start.month:02d}_to_{end.year}-{end.month:02d}.parquet')

transformed = transform_to_ts_data(validated)

In [ ]:
save_data(transformed, STREAM_DATA_DIR_SILVER, f'ts_data_{start.year}-{start.month:02d}_to_{end.year}-{end.month:02d}.parquet')

In [ ]:
ts_data = pd.read_parquet(INPUT_SILVER_DIR)

# string to datetime
ts_data['pickup_time'] = pd.to_datetime(ts_data['pickup_time'], utc=True)

# add column with Unix epoch milliseconds
ts_data['pickup_ts'] = ts_data['pickup_time'].astype(int) // 10**6

# Convert the pickup_location_id column to bigint
ts_data['pickup_location_transformed'] = ts_data['pickup_location_id'].astype('int64')

In [ ]:
# Connect to the project
project = hopsworks.login(
    project=cfg.HOPSWORKS_PROJECT_NAME,
    api_key_value=cfg.HOPSWORKS_API_KEY
)

# Connect to the feature store
feature_store = project.get_feature_store()

In [ ]:
# Connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=cfg.FEATURE_GROUP_NAME,
    version=cfg.FEATURE_GROUP_VERSION,
    description="ts data hourly frequency",
    primary_key = ['pickup_location_transformed', 'pickup_ts'],
    event_time='pickup_ts',
)

In [ ]:
# Insert features into the feature group
feature_group.insert(ts_data, write_options={"wait_for_job": False},
                        overwrite=True)